<a href="https://colab.research.google.com/github/abhishekworkspace/SmartEnergy/blob/master/pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [0]:
import pandas as pd
import numpy as np
import time
import datetime as dt
from sklearn.cluster import KMeans  
import random
from math import sqrt
import matplotlib.pylab as plt
import seaborn as sns

In [0]:
def toTimeStamp(date):
    for fmt in ('%d-%m-%Y %H:%M:%S', '%d-%m-%Y %H:%M', '%d/%m/%Y %H:%M'):
        try:
            return int(time.mktime(dt.datetime.strptime(date, fmt).timetuple()))
        except ValueError:
            pass

In [0]:
def toStrTime(timestamp):
    return time.strftime("%d/%m/%Y %H:%M", dt.datetime.fromtimestamp(timestamp).timetuple())

In [0]:
def toDateTime(timestamp):
    return pd.to_datetime(timestamp, unit='s') + pd.Timedelta('05:30:00')

In [0]:
df = pd.read_csv("Energy.csv")
df["Time"] = df["Time"].apply(toTimeStamp)

initialValue = toTimeStamp("31/12/2017 11:15")
finalValue = toTimeStamp("1/1/2019 00:00")

df2018 = df[df.apply(lambda x: initialValue <= x[0] < finalValue, axis = 1)]

newIndex =  pd.Index( np.arange(initialValue, finalValue, 15*60), name= "Time")
finalDf = df2018.set_index("Time").reindex(newIndex).reset_index()

FileNotFoundError: ignored

In [0]:
## Missing Value Interpretation
for i in range(2, finalDf.shape[0]):
    if(pd.isna(finalDf.loc[i, "Energy"])):
        if(pd.isna(finalDf.loc[i+1, "Energy"])):
          finalDf.loc[i, "Energy"] = np.average([finalDf.loc[i-1, "Energy"], finalDf.loc[i-2, "Energy"]])
        else:
          finalDf.loc[i, "Energy"] = np.average([finalDf.loc[i-1, "Energy"], finalDf.loc[i+1, "Energy"]])

finalDf = finalDf[finalDf.Time.apply(lambda x: x >= toTimeStamp("1/1/2018 00:00"))]
finalDf.head()

In [0]:
finalDf['Time'] = pd.to_datetime(finalDf['Time'], unit='s')
finalDf.head()

In [0]:
finalDf.set_index('Time',inplace=True)

In [0]:
## Normalization
finalDf.Time = finalDf.Time.apply(toDateTime)
dfMin = finalDf.Energy.min()
dfMax = finalDf.Energy.max()
finalDf.Energy = finalDf.Energy.apply(lambda x: (x - dfMin)/(dfMax - dfMin))
finalDf.Energy.head()

In [0]:
#### Outlier Detection
outlierValue = finalDf["Energy"].mean() + 2*finalDf["Energy"].std()
def replace(value):
   return outlierValue if(value > outlierValue) else value
finalDf.Energy = finalDf.Energy.apply(lambda x: outlierValue if(x > outlierValue) else x)
finalDf.Energy.head()

In [0]:
finalDf['dayofyear'] = finalDf.index.dayofyear
finalDf['month'] = finalDf.index.month
finalDf['date'] = finalDf.index.date
finalDf['dayofweek'] = finalDf.index.dayofweek + 1
finalDf['hourofday'] = finalDf.index.hour
finalDf['minuteofday'] = finalDf.index.minute
finalDf['daytype'] = ((pd.DatetimeIndex(finalDf.index).dayofweek) // 5 == 1).astype(int)

In [0]:
def assign_season(temp_df):
    if temp_df['month'] in [10,11,12,1,2]:
        return 1
    elif temp_df['month'] in [3,4,5,6]:
        return 2
    elif temp_df['month'] in [7,8,9]:
        return 3

In [0]:
finalDf['season'] = finalDf.apply(assign_season, axis=1)
finalDf.head()

In [0]:
finalDf.to_csv('Normalized 2018.csv')

In [0]:
## Normalization
finalDf.Time = finalDf.Time.apply(toDateTime)
dfMin = finalDf.Energy.min()
dfMax = finalDf.Energy.max()
finalDf.Energy = finalDf.Energy.apply(lambda x: (x - dfMin)/(dfMax - dfMin))
finalDf.Energy.head()

In [0]:
cl = pd.read_csv("Cluster 2\\Cluster2n.csv")
cl.head()

In [0]:
clu = cl.loc[cl['month'].isin(['3', '5'])]
clu.to_csv('Cluster3n.csv')

In [0]:
clu = cl.loc[cl['dayofweek'].isin(['1'])]
clu.to_csv('Cluster2Mondayn.csv')